In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Process 관련 변수
df_process_raw = pd.read_csv("C:/python/데이터셋/bat_process.csv", encoding ='euc-kr', low_memory=False)
# 시간 관련 변수
df_tat_raw=pd.read_csv("C:/python/데이터셋/bat_tat.csv", encoding='euc-kr', low_memory=False)

In [4]:
df_process_raw.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,37연,04단,3758.2,5,6.8,4696,37570,91,605,36


In [5]:
df_process_raw.describe(include='all')

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
count,43754,43754,43754,43754,43754,43754.000000,43754,43754,43754,43754,...,43727,43727,43754,43754,43754,43754,43754,43754,43754,43754
unique,45,1766,1766,43754,2,NaN,8,65,11,3,...,65,13,87,36,33,155,88,15,224,940
top,LOT-135,TRAY-1901,12-09-2021 오후 1:12:05,CELL-10747,양품,NaN,05열,38연,01단,03열,...,50연,03단,3759.5,5,6.8,4749,37587,92,580,107
freq,999,25,25,1,42379,NaN,6671,1141,5479,15594,...,1160,4314,2158,11501,9110,817,2687,10925,917,233
mean,NaN,NaN,NaN,NaN,NaN,12.925493,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,7.183505,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,7.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cell_id는 중복되지 않고, tray_id는 총 1766개

- total은 해당 tray의 cell 개수
- error은 해당 tray에서 불량인 cell의 개수
- rate는 불량률

In [21]:
# 불량률 계산 함수
def func1(total, error) :
    rate= ((error/total)*100).round(2)
    return rate

## 트레이 별 불량, 양품 개수 확인 : df_error_cnt

In [4]:
# 트레이 별 불량 양품 개수 확인
df_error_cnt=df_process_raw.groupby(['tray_id', 'judge'], as_index=False).size()
df_error_cnt

,tray_id,judge,size
0,TRAY-1002,양품,25
1,TRAY-1003,양품,25
2,TRAY-1004,양품,25
3,TRAY-1005,양품,25
4,TRAY-1006,양품,25
...,...,...,...
2188,TRAY-2789,양품,22
2189,TRAY-2790,불량,1
2190,TRAY-2790,양품,23
2191,TRAY-2791,불량,5


## 트레이 별 셀 개수 확인 : df_tray_cnt

In [6]:
# tray 별 cell 개수 확인
df_tray_cnt=df_process_raw.groupby('tray_id', as_index=False)['cell_id'].count()
df_tray_cnt.rename(columns={'cell_id': 'tray_cnt'}, inplace=True)
df_tray_cnt.head()

,tray_id,tray_cnt
0,TRAY-1002,25
1,TRAY-1003,25
2,TRAY-1004,25
3,TRAY-1005,25
4,TRAY-1006,25


## 트레이 별 셀 개수, 불량/양품 개수 출력 - df_error_cnt랑 df_tray_cnt merge

- tray_id : 트레이 아이디 <br>
- judge : 불량, 양품 판정 <br>
- size : 불량, 양품 개수 확인 <br>
- tray_cnt : tray 당 cell 개수

In [10]:
df_tot=pd.merge(df_error_cnt, df_tray_cnt, how='left', on='tray_id')
df_tot.head()

,tray_id,judge,size,tray_cnt
0,TRAY-1002,양품,25,25
1,TRAY-1003,양품,25,25
2,TRAY-1004,양품,25,25
3,TRAY-1005,양품,25,25
4,TRAY-1006,양품,25,25


In [11]:
df_tot.describe()

,size,tray_cnt
count,2193.000000,2193.000000
mean,19.951664,24.790242
std,8.645515,0.539292
min,1.000000,22.000000
25%,22.000000,25.000000
50%,25.000000,25.000000
75%,25.000000,25.000000
max,25.000000,25.000000


describe 결과 tray_cnt에서 최솟값은 22, 최댓값은 25확인. <br>
따라서 tray 당 cell 개수 최소 22개, 최대 25개 

## tray당 cell이 22개 일때

In [12]:
# tray 안에 cell이 22개 있을 때
df_tot[df_tot['tray_cnt']==22]

,tray_id,judge,size,tray_cnt
342,TRAY-1272,양품,22,22
382,TRAY-1304,양품,22,22
753,TRAY-1613,양품,22,22
760,TRAY-1619,양품,22,22
804,TRAY-1655,양품,22,22
934,TRAY-1755,양품,22,22
937,TRAY-1758,양품,22,22
940,TRAY-1760,불량,5,22
941,TRAY-1760,양품,17,22
1091,TRAY-1879,양품,22,22


In [18]:
df_tray_cnt[df_tray_cnt['tray_cnt']==22].shape

(15, 2)

따라서 cell 22개인 tray는 총 15개.

In [20]:
df_tot[(df_tot['tray_cnt']==22) & (df_tot['judge']=='불량')]

,tray_id,judge,size,tray_cnt
940,TRAY-1760,불량,5,22


cell이 22개인 tray중 불량 판정받은 tray는 tray-1760으로 1개 <br>

In [22]:
func1(df_tot[(df_tot['tray_cnt']==22) & (df_tot['judge']=='불량')]['tray_cnt'], df_tot[(df_tot['tray_cnt']==22) & (df_tot['judge']=='불량')]['size'])

940    22.73
dtype: float64

따라서 cell이 22개인 tray에서 불량률은 22.73% (1개 밖에 없음)

## tray당 cell이 23개일 때

In [25]:
df_tray_cnt[df_tray_cnt['tray_cnt']==23].shape

(79, 2)

cell이 23개인 tray총 79개

In [26]:
df_23=df_tot[(df_tot['tray_cnt']==23) & (df_tot['judge']=='불량')]
df_23.head()

,tray_id,judge,size,tray_cnt
46,TRAY-1043,불량,1,23
175,TRAY-1145,불량,2,23
286,TRAY-1232,불량,1,23
713,TRAY-1580,불량,1,23
781,TRAY-1635,불량,1,23


In [27]:
df_23['rate']=func1(df_tot[(df_tot['tray_cnt']==23) & (df_tot['judge']=='불량')]['tray_cnt'], df_tot[(df_tot['tray_cnt']==23) & (df_tot['judge']=='불량')]['size'])

In [28]:
df_23

,tray_id,judge,size,tray_cnt,rate
46,TRAY-1043,불량,1,23,4.35
175,TRAY-1145,불량,2,23,8.70
286,TRAY-1232,불량,1,23,4.35
713,TRAY-1580,불량,1,23,4.35
781,TRAY-1635,불량,1,23,4.35
866,TRAY-1705,불량,1,23,4.35
967,TRAY-1780,불량,1,23,4.35
1113,TRAY-1900,불량,1,23,4.35
1198,TRAY-1976,불량,4,23,17.39
1922,TRAY-2545,불량,1,23,4.35


rate는 각 tray에서의 불량률

## tray당 cell이 24개일 때

In [29]:
df_tray_cnt[df_tray_cnt['tray_cnt']==24].shape

(193, 2)

cell이 24개인 tray 총 193개

In [30]:
df_24=df_tot[(df_tot['tray_cnt']==24) & (df_tot['judge']=='불량')]

In [32]:
df_24['rate']=func1(df_tot[(df_tot['tray_cnt']==24) & (df_tot['judge']=='불량')]['tray_cnt'], df_tot[(df_tot['tray_cnt']==24) & (df_tot['judge']=='불량')]['size'])

In [33]:
df_24.head()

,tray_id,judge,size,tray_cnt,rate
35,TRAY-1034,불량,1,24,4.17
84,TRAY-1073,불량,1,24,4.17
216,TRAY-1173,불량,1,24,4.17
233,TRAY-1186,불량,5,24,20.83
281,TRAY-1228,불량,1,24,4.17


## tray당 cell이 25개일 때

In [34]:
df_tray_cnt[df_tray_cnt['tray_cnt']==25].shape

(1479, 2)

cell이 25개인 tray총 1479개

In [36]:
df_25=df_tot[(df_tot['tray_cnt']==25) & (df_tot['judge']=='불량')]

In [37]:
df_25['rate']=func1(df_tot[(df_tot['tray_cnt']==25) & (df_tot['judge']=='불량')]['tray_cnt'], df_tot[(df_tot['tray_cnt']==25) & (df_tot['judge']=='불량')]['size'])

In [38]:
df_25

,tray_id,judge,size,tray_cnt,rate
7,TRAY-1009,불량,1,25,4.0
11,TRAY-1012,불량,1,25,4.0
24,TRAY-1024,불량,2,25,8.0
42,TRAY-1040,불량,1,25,4.0
49,TRAY-1045,불량,2,25,8.0
...,...,...,...,...,...
2168,TRAY-2768,불량,1,25,4.0
2172,TRAY-2772,불량,2,25,8.0
2185,TRAY-2788,불량,6,25,24.0
2187,TRAY-2789,불량,3,25,12.0


In [39]:
df_25.describe(include='all')

,tray_id,judge,size,tray_cnt,rate
count,375,375,375.000000,375.0,375.000000
unique,375,1,NaN,NaN,NaN
top,TRAY-1009,불량,NaN,NaN,NaN
freq,1,375,NaN,NaN,NaN
mean,NaN,NaN,3.256000,25.0,13.024000
std,NaN,NaN,3.764961,0.0,15.059844
min,NaN,NaN,1.000000,25.0,4.000000
25%,NaN,NaN,1.000000,25.0,4.000000
50%,NaN,NaN,2.000000,25.0,8.000000
75%,NaN,NaN,4.000000,25.0,16.000000


In [40]:
df_25[df_25['rate']>30]

,tray_id,judge,size,tray_cnt,rate
98,TRAY-1085,불량,8,25,32.0
162,TRAY-1133,불량,13,25,52.0
299,TRAY-1240,불량,14,25,56.0
306,TRAY-1245,불량,8,25,32.0
334,TRAY-1267,불량,12,25,48.0
524,TRAY-1425,불량,8,25,32.0
526,TRAY-1426,불량,16,25,64.0
532,TRAY-1430,불량,15,25,60.0
536,TRAY-1433,불량,8,25,32.0
538,TRAY-1434,불량,22,25,88.0


In [42]:
df_process_raw.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,37연,04단,3758.2,5,6.8,4696,37570,91,605,36


In [43]:
# tray 별 cell 개수 확인
df_tray_cnt=df_process_raw.groupby('tray_id', as_index=False)['cell_id'].count()
df_tray_cnt.rename(columns={'cell_id': 'tray_cnt'}, inplace=True)
df_tray_cnt.head()

,tray_id,tray_cnt
0,TRAY-1002,25
1,TRAY-1003,25
2,TRAY-1004,25
3,TRAY-1005,25
4,TRAY-1006,25


In [44]:
df_tray_cnt.shape

(1766, 2)

In [69]:
df_cnt = df_process_raw[['tray_id', 'judge']].groupby(['tray_id', 'judge'], as_index=False).size()

In [70]:
import copy
df_process = copy.deepcopy(df_process_raw)
df_process.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,37연,04단,3758.2,5,6.8,4696,37570,91,605,36


In [71]:
df_process

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,37연,04단,3758.2,5,6.8,4696,37570,91,605,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53649,양품,16,08열,21연,03단,02열,...,16연,13단,3759.2,5.3,7.2,4745,37583,94,643,-35
43750,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53650,양품,14,08열,21연,03단,02열,...,16연,13단,3759.2,5.3,7.1,4746,37583,94,620,89
43751,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53651,양품,2,08열,21연,03단,02열,...,16연,13단,3760.3,5.2,7,4782,37595,92,597,-23
43752,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-54355,양품,15,08열,21연,03단,02열,...,16연,13단,3760,5.3,7,4751,37590,95,620,3


In [72]:
df_cnt

,tray_id,judge,size
0,TRAY-1002,양품,25
1,TRAY-1003,양품,25
2,TRAY-1004,양품,25
3,TRAY-1005,양품,25
4,TRAY-1006,양품,25
...,...,...,...
2188,TRAY-2789,양품,22
2189,TRAY-2790,불량,1
2190,TRAY-2790,양품,23
2191,TRAY-2791,불량,5


In [73]:
df_cnt = df_cnt[df_cnt['judge']=='불량'][['tray_id', 'size']]

In [80]:
df_cnt

,tray_id,size
7,TRAY-1009,1
11,TRAY-1012,1
24,TRAY-1024,2
35,TRAY-1034,1
42,TRAY-1040,1
...,...,...
2183,TRAY-2787,4
2185,TRAY-2788,6
2187,TRAY-2789,3
2189,TRAY-2790,1


In [74]:
df_process = pd.merge(df_process, df_cnt, how='left', on='tray_id')

In [78]:
df_process.fillna(0, inplace=True)

In [79]:
df_process

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,size
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,04단,3759.3,5,6.8,4718,37583,94,565,164,0.0
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,04단,3758.4,5,6.8,4718,37574,94,565,73,0.0
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,04단,3759,5.1,6.9,4700,37579,91,585,-57,0.0
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,04단,3759.5,5.1,6.9,4688,37583,93,606,99,0.0
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,04단,3758.2,5,6.8,4696,37570,91,605,36,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53649,양품,16,08열,21연,03단,02열,...,13단,3759.2,5.3,7.2,4745,37583,94,643,-35,5.0
43750,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53650,양품,14,08열,21연,03단,02열,...,13단,3759.2,5.3,7.1,4746,37583,94,620,89,5.0
43751,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53651,양품,2,08열,21연,03단,02열,...,13단,3760.3,5.2,7,4782,37595,92,597,-23,5.0
43752,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-54355,양품,15,08열,21연,03단,02열,...,13단,3760,5.3,7,4751,37590,95,620,3,5.0


In [83]:
df_process = pd.merge(df_process, df_tray_cnt, how='left', on='tray_id')

In [84]:
df_process

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,size,tray_cnt
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,3759.3,5,6.8,4718,37583,94,565,164,0.0,24
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,3758.4,5,6.8,4718,37574,94,565,73,0.0,24
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,3759,5.1,6.9,4700,37579,91,585,-57,0.0,24
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,3759.5,5.1,6.9,4688,37583,93,606,99,0.0,24
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,3758.2,5,6.8,4696,37570,91,605,36,0.0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53649,양품,16,08열,21연,03단,02열,...,3759.2,5.3,7.2,4745,37583,94,643,-35,5.0,25
43750,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53650,양품,14,08열,21연,03단,02열,...,3759.2,5.3,7.1,4746,37583,94,620,89,5.0,25
43751,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53651,양품,2,08열,21연,03단,02열,...,3760.3,5.2,7,4782,37595,92,597,-23,5.0,25
43752,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-54355,양품,15,08열,21연,03단,02열,...,3760,5.3,7,4751,37590,95,620,3,5.0,25


In [86]:
df_process['rate']=((df_process['size']/df_process['tray_cnt'])*100).round(2)

In [87]:
df_process

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,size,tray_cnt,rate
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,5,6.8,4718,37583,94,565,164,0.0,24,0.0
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,5,6.8,4718,37574,94,565,73,0.0,24,0.0
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,5.1,6.9,4700,37579,91,585,-57,0.0,24,0.0
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,5.1,6.9,4688,37583,93,606,99,0.0,24,0.0
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,5,6.8,4696,37570,91,605,36,0.0,24,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53649,양품,16,08열,21연,03단,02열,...,5.3,7.2,4745,37583,94,643,-35,5.0,25,20.0
43750,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53650,양품,14,08열,21연,03단,02열,...,5.3,7.1,4746,37583,94,620,89,5.0,25,20.0
43751,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53651,양품,2,08열,21연,03단,02열,...,5.2,7,4782,37595,92,597,-23,5.0,25,20.0
43752,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-54355,양품,15,08열,21연,03단,02열,...,5.3,7,4751,37590,95,620,3,5.0,25,20.0


In [88]:
df_process.drop(['size', 'tray_cnt'], axis=1, inplace=True)

In [89]:
df_process

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rate
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,04단,3759.3,5,6.8,4718,37583,94,565,164,0.0
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,04단,3758.4,5,6.8,4718,37574,94,565,73,0.0
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,04단,3759,5.1,6.9,4700,37579,91,585,-57,0.0
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,04단,3759.5,5.1,6.9,4688,37583,93,606,99,0.0
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,04단,3758.2,5,6.8,4696,37570,91,605,36,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53649,양품,16,08열,21연,03단,02열,...,13단,3759.2,5.3,7.2,4745,37583,94,643,-35,20.0
43750,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53650,양품,14,08열,21연,03단,02열,...,13단,3759.2,5.3,7.1,4746,37583,94,620,89,20.0
43751,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-53651,양품,2,08열,21연,03단,02열,...,13단,3760.3,5.2,7,4782,37595,92,597,-23,20.0
43752,LOT-144,TRAY-2791,12-17-2021 오후 6:33:15,CELL-54355,양품,15,08열,21연,03단,02열,...,13단,3760,5.3,7,4751,37590,95,620,3,20.0


In [93]:
df_process.to_csv('df_rate.csv', encoding='euc-kr')